In [1]:
import numpy as np
import pandas as pd

In [10]:
csv_file = "matches_2020-07-17T06:55:48.483660+00:00.csv"
csv_data = pd.read_csv(csv_file)

In [5]:
csv_data.head()

,timestamp,ais_distance,ais_bearing,ais_width,ais_length,ai_bearing,imageIndex,y_max_rel,height,height_rel,width,width_rel,vessel_course,mode,match
0,2020-06-28T17:33:00+00:00,2354.8,263.0,16.0,128.0,268.12,2,55.6,19.0,9.5,19.0,1.1,279.2,open_sea,1
1,2020-06-28T18:06:00+00:00,984.9,205.7,16.0,128.0,215.27,1,61.5,29.0,14.4,95.0,5.3,257.1,open_sea,1
2,2020-06-25T06:09:00+00:00,5293.3,82.4,16.0,140.0,87.18,2,52.6,8.0,4.0,14.0,0.8,99.2,open_sea,1
3,2020-06-25T01:46:00+00:00,3555.2,75.9,28.0,185.0,77.56,3,54.1,12.0,6.0,10.0,0.5,71.3,open_sea,1
4,2020-06-25T01:46:00+00:00,19032.5,33.4,24.0,180.0,42.90,2,50.6,8.0,4.0,11.0,0.6,71.3,open_sea,1


In [11]:
csv_data["bearing_diff"] = csv_data["ai_bearing"] - csv_data["ais_bearing"]
csv_data["height_rel"] = csv_data["height_rel"] / 100
csv_data["width_rel"] = csv_data["width_rel"] / 100
csv_data["y_max_rel"] = csv_data["y_max_rel"] / 100
csv_data["image_1"] = (csv_data["imageIndex"] == 1).astype(int)
csv_data["image_2"] = (csv_data["imageIndex"] == 2).astype(int)
csv_data["image_3"] = (csv_data["imageIndex"] == 3).astype(int)
csv_data["image_4"] = (csv_data["imageIndex"] == 4).astype(int)
csv_data["image_5"] = (csv_data["imageIndex"] == 5).astype(int)

def normalize_bearing(r):
    bearing = r["ais_bearing"]
    if bearing <= 180:
        return bearing
    return bearing - 360

csv_data["ais_bearing"] = csv_data.apply(normalize_bearing, axis=1)

X = csv_data.drop(columns=["timestamp", "ai_bearing", "mode", "height", "width", "imageIndex"])
y = csv_data["match"]

X.head()

,ais_distance,ais_bearing,ais_width,ais_length,y_max_rel,height_rel,width_rel,vessel_course,match,bearing_diff,image_1,image_2,image_3,image_4,image_5
0,2354.8,-97.0,16.0,128.0,0.556,0.095,0.011,279.2,1,5.12,0,1,0,0,0
1,984.9,-154.3,16.0,128.0,0.615,0.144,0.053,257.1,1,9.57,1,0,0,0,0
2,5293.3,82.4,16.0,140.0,0.526,0.040,0.008,99.2,1,4.78,0,1,0,0,0
3,3555.2,75.9,28.0,185.0,0.541,0.060,0.005,71.3,1,1.66,0,0,1,0,0
4,19032.5,33.4,24.0,180.0,0.506,0.040,0.006,71.3,1,9.50,0,1,0,0,0


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)

X_test.head()

accuracy_score(y_test.values, y_predict)

1.0